In [ ]:
import os
import soundfile as sf
from pydub import AudioSegment
import librosa
import numpy as np

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical
from scipy.io.wavfile import write
# import tensorflow as tf

import matplotlib.pyplot as plt
import cv2

import colorednoise as cn

In [ ]:
def manipulate(data, noise_factor):
    pink_noise = cn.powerlaw_psd_gaussian(1, data.shape[0])
    augmented_data = data + noise_factor * pink_noise
    # Cast back to same data type
    augmented_data = augmented_data.astype(type(data[0]))
    return augmented_data

In [ ]:
desiredSampleRate = 16000
for directory in ["up", "down", "left", "right"]:
    for d in os.listdir("./dataset/proccessed/" + directory):
           
        path_to_file = "/mnt/g/onlinelessons/Multimedia/HW/project/dataset/proccessed/" + directory + '/'+ d
        
        output_filepath = os.path.splitext(path_to_file)[0] + '.wav'
        output_filepath_n = os.path.splitext(path_to_file)[0] + "_noise" +  '.wav'
        
        audio = AudioSegment.from_file(path_to_file)
        # Set the sample width to 2 bytes (16-bit)
        audio = audio.set_sample_width(2)
        
        audio.export(output_filepath, format='wav')
        
        audio, sr = librosa.load(output_filepath, sr= desiredSampleRate, mono=True)

        audio_n = manipulate(audio, 0.00001)
        write(output_filepath_n, desiredSampleRate, audio_n)
        audio = librosa.to_mono(audio)
        librosa.resample(audio, sr, desiredSampleRate)
        audio = librosa.util.normalize(audio)
        audio = audio.reshape(-1, 1)    
        
        sf.write(output_filepath, audio, sr, 'PCM_24')
        sf.write(output_filepath_n, audio_n, sr, 'PCM_24')